# 作成したモデルでの正誤を一覧にする

一旦モデル設計とのコードと分離したかったため、.h5ファイルを読み込んで使用する。

In [66]:
import os
import re
# CPUの利用を強制する場合
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import shutil
import random
import tensorflow as tf
import keras
from keras import Sequential
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from keras.optimizers import SGD

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# 環境変数とか

# 読み込むモデル
model_path = '.\\models\\vgg16-tuning-150epoch.h5'


# 元データ保存先
dataset_base_path = '.\\splat-scene-dataset'
dataset_split_base_path = '.\\dataset'
tensorboard_log_path = '.\\tflog'

# 画像設定
input_size = (640, 360)
input_shape = (640, 360, 3)

# データ関係
batch_size = 1 # 1回でいい
categories_n = 17

dataset_train_path = os.path.join(dataset_split_base_path, 'train')
dataset_val_path   = os.path.join(dataset_split_base_path, 'val')
dataset_test_path  = os.path.join(dataset_split_base_path, 'test')
pathes = [dataset_train_path, dataset_val_path, dataset_test_path]

In [6]:
def load_model(path):
    return keras.models.load_model(path)

model = load_model(model_path)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 640, 360, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 640, 360, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 640, 360, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 320, 180, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 320, 180, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 320, 180, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 160, 90, 128)      0         
__________

In [7]:
# ImageDataGeneratorを生成
def create_generator(path,
                     target_size = (640, 360),
                     batch_size = 1,
                     class_mode = 'categorical'):
    print(path)
    dg = ImageDataGenerator(rescale=1/255.0)
    gen = dg.flow_from_directory(path, 
                                 target_size=target_size,
                                 batch_size=batch_size,
                                 class_mode=class_mode,
                                 shuffle=False)
    return (dg, gen)

(train_dg, train_gen) = create_generator(dataset_train_path, target_size=input_size, batch_size=batch_size)
(val_dg,   val_gen)   = create_generator(dataset_val_path, target_size=input_size, batch_size=batch_size)
(test_dg,  test_gen)  = create_generator(dataset_test_path, target_size=input_size, batch_size=batch_size)

generators = [train_gen, val_gen, test_gen]

.\dataset\train
Found 10231 images belonging to 17 classes.
.\dataset\val
Found 3406 images belonging to 17 classes.
.\dataset\test
Found 3406 images belonging to 17 classes.


In [9]:
predicts = [model.predict_generator(g, len(g.filenames), verbose=1) for g in generators]
predicts

 7756/10231 [=====================>........] - ETA: 52:35

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2829/3406 [=======================>......] - ETA: 11:43

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [71]:
# ファイルごとに予測結果を出力させる
def create_result(generators, predicts):
    # label_name: indexのdictなので、順に並び替えて使えるようにしておく
    labels = [x[0] for x in sorted(generators[0].class_indices.items(), key=lambda x: x[1])]
    def generate(generators, predicts):
        for g,ps in zip(generators, predicts):
            files = g.filenames
            for file, predict in zip(files, ps):
                # 基本譲歩を追加
                dst = {'base_dir': g.directory, 'file': file, 'path': os.path.join(g.directory, file), }
                # 予測情報を追加
                for label, p in zip(labels, predict):
                    dst['predict_{}'.format(label)] = p
                # 予測結果が一致しているかを追加
                index = predict.argmax()
                label_correct = re.search(r'[^\\]+', file).group(0)
                label_predict = labels[index]
                is_correct = label_predict == label_correct
                
                dst['label_predict'] = label_predict
                dst['label_correct'] = label_correct
                dst['is_correct'] = is_correct
                yield dst
    return pd.DataFrame(generate(generators, predicts))

result = create_result(generators, predicts)

# 結果をCSV保存しとく
result.to_csv('predict_all.csv')
result[result['is_correct'] == False].to_csv('predict_miss.csv')

result

,base_dir,file,is_correct,label_correct,label_predict,path,predict_battle,predict_battle_finish,predict_battle_loby,predict_battle_matching,...,predict_loading,predict_menu,predict_other,predict_salmon,predict_salmon_lobby,predict_salmon_matching,predict_salmon_miss,predict_salmon_result,predict_salmon_start,predict_weapon_select
0,.\dataset\train,battle\WIN_20181022_15_12_23_Pro-00000044.jpg,True,battle,battle,.\dataset\train\battle\WIN_20181022_15_12_23_P...,0.997399,0.000037,4.884951e-07,2.019877e-11,...,1.370866e-07,1.511804e-07,0.000002,0.000524,0.000001,6.138729e-09,0.000002,0.000004,0.000084,2.529286e-07
1,.\dataset\train,battle\WIN_20181022_15_12_23_Pro-00000046.jpg,True,battle,battle,.\dataset\train\battle\WIN_20181022_15_12_23_P...,0.985718,0.000554,7.004309e-05,2.908166e-07,...,4.669232e-05,1.552678e-05,0.000073,0.010465,0.000069,4.137493e-06,0.000130,0.000357,0.000899,1.818288e-05
2,.\dataset\train,battle\WIN_20181022_15_12_23_Pro-00000049.jpg,True,battle,battle,.\dataset\train\battle\WIN_20181022_15_12_23_P...,0.977944,0.002013,7.031678e-05,5.699937e-08,...,1.726501e-05,2.445622e-05,0.000107,0.003270,0.000100,4.462861e-06,0.000162,0.000455,0.001721,3.592077e-05
3,.\dataset\train,battle\WIN_20181022_15_12_23_Pro-00000050.jpg,True,battle,battle,.\dataset\train\battle\WIN_20181022_15_12_23_P...,0.977312,0.001208,4.469664e-05,1.245774e-08,...,8.840407e-06,1.598853e-05,0.000083,0.005445,0.000068,1.953742e-06,0.000106,0.000309,0.001245,4.093720e-05
4,.\dataset\train,battle\WIN_20181022_15_12_23_Pro-00000051.jpg,True,battle,battle,.\dataset\train\battle\WIN_20181022_15_12_23_P...,0.873103,0.008971,2.086634e-04,3.560338e-07,...,1.769113e-04,9.583623e-05,0.000552,0.015909,0.000497,2.253660e-05,0.000899,0.003348,0.010410,1.233413e-04
5,.\dataset\train,battle\WIN_20181022_15_12_23_Pro-00000053.jpg,True,battle,battle,.\dataset\train\battle\WIN_20181022_15_12_23_P...,0.992605,0.000211,3.342786e-06,4.480570e-10,...,5.423142e-07,1.287508e-06,0.000010,0.001298,0.000007,8.969344e-08,0.000013,0.000028,0.000252,2.238736e-06
6,.\dataset\train,battle\WIN_20181022_15_12_23_Pro-00000058.jpg,True,battle,battle,.\dataset\train\battle\WIN_20181022_15_12_23_P...,0.989002,0.000641,9.964325e-06,1.635641e-08,...,3.991962e-06,6.773340e-06,0.000037,0.000799,0.000031,9.677710e-07,0.000055,0.000166,0.000676,1.000894e-05
7,.\dataset\train,battle\WIN_20181022_15_12_23_Pro-00000059.jpg,True,battle,battle,.\dataset\train\battle\WIN_20181022_15_12_23_P...,0.957054,0.002287,5.188542e-05,9.253609e-09,...,7.351238e-06,1.701830e-05,0.000092,0.023785,0.000076,2.254527e-06,0.000125,0.000268,0.001721,4.029189e-05
8,.\dataset\train,battle\WIN_20181022_15_12_23_Pro-00000061.jpg,True,battle,battle,.\dataset\train\battle\WIN_20181022_15_12_23_P...,0.968667,0.005943,2.633369e-04,8.382802e-07,...,8.539885e-05,9.420123e-05,0.000309,0.005701,0.000303,2.903181e-05,0.000512,0.001571,0.002721,1.297771e-04
9,.\dataset\train,battle\WIN_20181022_15_12_23_Pro-00000063.jpg,True,battle,battle,.\dataset\train\battle\WIN_20181022_15_12_23_P...,0.992858,0.001206,8.136284e-05,1.464555e-07,...,1.388300e-05,2.043509e-05,0.000072,0.002090,0.000066,7.619834e-06,0.000105,0.000239,0.000821,2.488750e-05


In [72]:
# 結果をいろいろ図示してあげる